In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# Step 1: Create dataset
data = {
    'x1': [1,2,3,4,5,6,7,8,9,10],
    'x2': [2,1,3,4,5,2,4,6,5,7],
    'x3': [3,2,2,4,5,3,3,5,4,6],
    'x4': [4,3,5,2,5,1,5,4,2,5],
    'x5': [5,4,1,3,5,2,1,3,3,4],
    'y':  [35,30,40,50,75,40,55,70,65,85]
}
df = pd.DataFrame(data)

# Step 2: Separate X and y
X = df[['x1', 'x2', 'x3', 'x4', 'x5']]
y = df['y']

# Step 3: Train linear regression
model = LinearRegression()
model.fit(X, y)

# Step 4: Show slope (weights) and intercept
print("Weights (slopes):", model.coef_)
print("Intercept:", model.intercept_)

# Step 5: Check predicted vs actual
df['predicted'] = model.predict(X)
print(df)


Weights (slopes): [1.8728317  3.82307485 4.05328578 1.82439386 1.28966365]
Intercept: 3.7265011729122435
   x1  x2  x3  x4  x5   y  predicted
0   1   2   3   4   5  35  39.151234
1   2   1   2   3   4  30  30.033647
2   3   3   2   5   1  40  39.332425
3   4   4   4   2   3  50  50.241049
4   5   5   5   5   5  75  68.042750
5   6   2   3   1   2  40  39.173220
6   7   4   3   5   1  55  54.700113
7   8   6   5   4   3  70  73.080599
8   9   5   4   2   3  65  63.428282
9  10   7   6   5   4  85  87.816681
